### Data Preparation

In [1]:
# Prepare the data so that the target includes both stock and bond returns.

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset
import torch

import torch.optim as optim
import torch.nn as nn

# Example DataFrame creation
data = {
    'stock_return': np.random.randn(100),
    'bond_return': np.random.randn(100),
    'CPI': np.random.randn(100),
    'future_stock_return': np.random.randn(100),
    'future_bond_return': np.random.randn(100)  # additional target variable
}
df = pd.DataFrame(data)

# Normalize the features and targets
scaler_features = MinMaxScaler()
scaled_features = scaler_features.fit_transform(df[['stock_return', 'bond_return', 'CPI']])

scaler_target = MinMaxScaler()
scaled_target = scaler_target.fit_transform(df[['future_stock_return', 'future_bond_return']])

# Create sequences for LSTM input
def create_sequences(data, target, seq_length):
    sequences = []
    targets = []
    for i in range(len(data) - seq_length):
        seq = data[i:i+seq_length]
        label = target[i+seq_length]
        sequences.append(seq)
        targets.append(label)
    return np.array(sequences), np.array(targets)

SEQ_LENGTH = 10
X, y = create_sequences(scaled_features, scaled_target, SEQ_LENGTH)

# Convert to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)

dataset = TensorDataset(X_tensor, y_tensor)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)


### Model Definition

In [2]:

# Update the model to handle multiple outputs (both stock and bond returns).


class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        h0 = torch.zeros(num_layers, x.size(0), hidden_size).to(x.device)
        c0 = torch.zeros(num_layers, x.size(0), hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

input_size = 3  # number of features: stock_return, bond_return, CPI
hidden_size = 50
num_layers = 2
output_size = 2  # predicting both stock and bond returns

model = LSTMModel(input_size, hidden_size, num_layers, output_size)


### Training

In [3]:
# Train the model with the updated setup.
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 20

for epoch in range(num_epochs):
    for X_batch, y_batch in dataloader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [1/20], Loss: 0.2266
Epoch [2/20], Loss: 0.1136
Epoch [3/20], Loss: 0.0310
Epoch [4/20], Loss: 0.0386
Epoch [5/20], Loss: 0.0316
Epoch [6/20], Loss: 0.0395
Epoch [7/20], Loss: 0.0366
Epoch [8/20], Loss: 0.0306
Epoch [9/20], Loss: 0.0230
Epoch [10/20], Loss: 0.0410
Epoch [11/20], Loss: 0.0288
Epoch [12/20], Loss: 0.0271
Epoch [13/20], Loss: 0.0437
Epoch [14/20], Loss: 0.0477
Epoch [15/20], Loss: 0.0122
Epoch [16/20], Loss: 0.0481
Epoch [17/20], Loss: 0.0311
Epoch [18/20], Loss: 0.0366
Epoch [19/20], Loss: 0.0250
Epoch [20/20], Loss: 0.0174


### Prediction

In [4]:
# Make predictions for both stock and bond returns.

# To make predictions
model.eval()
with torch.no_grad():
    sample_input = torch.tensor(X[:1], dtype=torch.float32)
    prediction = model(sample_input)
    prediction = scaler_target.inverse_transform(prediction.numpy())  # Convert back to original scale
    print(prediction)


[[-0.02639892 -0.04806899]]


### Summary

In this updated example:

1. **Data Preparation**: Both `future_stock_return` and `future_bond_return` are included as targets.
2. **Model Definition**: The output size of the LSTM model is set to 2 to predict both returns.
3. **Training**: The training loop remains the same, but now the model predicts multiple targets.
4. **Prediction**: Predictions are scaled back to the original range for interpretability.

Make sure to adapt the `input_size`, `output_size`, and other parameters according to your specific dataset and requirements.